### Script para classificação dos relatórios

In [5]:
import fitz  # PyMuPDF
import pandas as pd
from pathlib import Path
import shutil


# Defina o caminho para a pasta principal onde seus relatórios estão.
DIR_ORIGEM = Path('data/pdf/yearly')
DIR_DESTINO = Path('data/pdf/categorized')

In [6]:
def classify_cenipa_report(pdf_path):
    """
    Analisa a primeira página de um PDF e o classifica em uma de três categorias.

    Args:
        pdf_path (Path): O caminho para o arquivo PDF.

    Returns:
        str: A categoria do relatório ('Simplificado', 'Completo', 'Imagem', 'Outro').
    """
    # Limite de caracteres para considerar um PDF como sendo baseado em imagem.
    IMAGE_TEXT_THRESHOLD = 100 # Aumentado um pouco para mais robustez

    try:
        doc = fitz.open(pdf_path)

        # Analisa apenas a primeira página para otimização
        if doc.page_count > 0:
            page = doc.load_page(0)
            text = page.get_text("text")
        else:
            # Arquivo PDF vazio ou corrompido
            return 'Erro'

        doc.close()

        # 1. Checar se é um relatório baseado em imagem
        if len(text.strip()) < IMAGE_TEXT_THRESHOLD:
            return 'Imagem'

        # Converte o texto para maiúsculas uma vez para otimizar as buscas
        upper_text = text.upper()

        # 2. Checar se é um Relatório Simplificado ou uma Suma de Investigação
        # Ambos serão classificados como 'Simplificado'
        if 'RELATÓRIO FINAL SIMPLIFICADO' in upper_text or 'SUMA DE INVESTIGAÇÃO' in upper_text:
            return 'Simplificado'

        # 3. Se não for nenhum dos anteriores, é um Relatório Completo
        # Relatórios completos geralmente contêm apenas "RELATÓRIO FINAL"
        if 'RELATÓRIO FINAL' in upper_text:
             return 'Completo'

        # Como último recurso, se nenhum padrão for encontrado, classificamos como 'Outro'
        # Isso ajuda a identificar PDFs que não se encaixam nos padrões esperados.
        return 'Outro'


    except Exception as e:
        print(f"Erro ao processar o arquivo {pdf_path}: {e}")
        return 'Erro'

In [7]:
print("Iniciando a varredura e classificação dos relatórios...")

# Encontra todos os arquivos .pdf recursivamente no diretório base
pdf_files = list(DIR_ORIGEM.rglob('*.pdf'))

if not pdf_files:
    print("\nAVISO: Nenhum arquivo PDF encontrado.")
    print(f"Certifique-se de que seus arquivos estão na pasta '{DIR_ORIGEM}'.")
else:
    results = []
    for file_path in pdf_files:
        category = classify_cenipa_report(file_path)
        results.append({
            'arquivo': file_path.name,
            'caminho': str(file_path),
            'categoria': category
        })

    # Converte os resultados para um DataFrame do Pandas
    df_results = pd.DataFrame(results)

    print(f"\nProcessamento concluído! {len(pdf_files)} arquivos analisados.")

    # Exibe as primeiras linhas da tabela de resultados
    print("\nAmostra dos resultados da classificação:")
    display(df_results.head())

    # Calcula as estatísticas
    stats = df_results['categoria'].value_counts()
    stats_percent = df_results['categoria'].value_counts(normalize=True) * 100

    print("\n--- Estatísticas Gerais ---")
    summary_df = pd.DataFrame({
        'Quantidade': stats,
        'Porcentagem (%)': stats_percent.round(2)
    })
    display(summary_df)

Iniciando a varredura e classificação dos relatórios...
MuPDF error: library error: zlib error: incorrect header check


Processamento concluído! 3008 arquivos analisados.

Amostra dos resultados da classificação:


,arquivo,caminho,categoria
0,pp_eam_31_01_00.pdf,data\pdf\yearly\2000\pp_eam_31_01_00.pdf,Outro
1,pp_efc_30_05_00.pdf,data\pdf\yearly\2000\pp_efc_30_05_00.pdf,Completo
2,pp_emv_24_08_00.pdf,data\pdf\yearly\2000\pp_emv_24_08_00.pdf,Outro
3,pp_eog_18_01_00.pdf,data\pdf\yearly\2000\pp_eog_18_01_00.pdf,Completo
4,pp_fju_03_09_00.pdf,data\pdf\yearly\2000\pp_fju_03_09_00.pdf,Completo



--- Estatísticas Gerais ---


,Quantidade,Porcentagem (%)
categoria,,
Simplificado,1599,53.16
Completo,1154,38.36
Outro,142,4.72
Imagem,111,3.69
Erro,2,0.07


In [8]:
print("Organizando arquivos em pastas por categoria...")

# Cria uma pasta para cada categoria que ele encontra
for category_name in df_results['categoria'].unique():
     category_dir = DIR_DESTINO / category_name
     category_dir.mkdir(parents=True, exist_ok=True)

# Coloca os arquivos na pasta criada
for index, row in df_results.iterrows():
    source_path = Path(row['caminho'])
    category = row['categoria']

    # Define the destination path based on the category
    destination_path = DIR_DESTINO / category / source_path.name

    try:
        # Check if source and destination are the same file to avoid errors when the file is already in the correct folder
        if source_path.resolve() == destination_path.resolve():
            print(f"   Pulando movimentação: {source_path.name} já está na pasta correta.")
            continue

        # Copia o arquivo para a pasta organizada
        shutil.copy2(source_path, destination_path)
        print(f"   Movido: {source_path.name} para '{category}'")
    except Exception as e:
        print(f"   Erro ao mover {source_path.name}: {e}")


print(f"\nArquivos movidos e organizados em '{DIR_DESTINO}'.")

Organizando arquivos em pastas por categoria...
   Movido: pp_eam_31_01_00.pdf para 'Outro'
   Movido: pp_efc_30_05_00.pdf para 'Completo'
   Movido: pp_emv_24_08_00.pdf para 'Outro'
   Movido: pp_eog_18_01_00.pdf para 'Completo'
   Movido: pp_fju_03_09_00.pdf para 'Completo'
   Movido: pp_gbi_22_07_00.pdf para 'Outro'
   Movido: pp_gcn_25_01_00.pdf para 'Outro'
   Movido: pp_gki_27_04_00.pdf para 'Completo'
   Movido: pp_gvq_21_01_00.pdf para 'Outro'
   Movido: pp_mll_11_11_00.pdf para 'Completo'
   Movido: pt_atq_08_04_00.pdf para 'Outro'
   Movido: pt_djb_19_08_00.pdf para 'Completo'
   Movido: pt_dog_10_05_00.pdf para 'Completo'
   Movido: pt_edy_25_08_00.pdf para 'Outro'
   Movido: pt_euf_03_03_00.pdf para 'Completo'
   Movido: pt_ghv_06_01_00.pdf para 'Completo'
   Movido: pt_hhf_09_11_00.pdf para 'Completo'
   Movido: pt_hns_02_02_00.pdf para 'Completo'
   Movido: pt_hon_03_02_00.pdf para 'Completo'
   Movido: pt_hoz_20_04_00.pdf para 'Outro'
   Movido: pt_hrd_02_08_00_1.pdf par